In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Get Visualisations of Each Model

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/content/drive/MyDrive/Projects/Nuclear Physics GPT/Feature Engineering/nuclear_physics_features_df.csv', index_col=0).reset_index(drop=True)

df.head(5)

,Protons,Protons_log,Protons_reciprocal,Neutrons,Neutrons_log,Neutrons_reciprocal,Neutrons_power,Atomic Weight,Atomic Weight_log,Atomic Weight_reciprocal,...,Last Azimuthal Quantum Number Encoded,Is Magic Number,Magic Number,Closest Magic Number,Distance To Magic Number,Distance To Magic Number_abs,Half-life,Half-life Magnitude,Half-life Time,Artificial
0,1,0.693147,1.0,0,0.000000,0.000000,0,1.007825,0.697052,0.992236,...,0,False,0,2,-1,1,2.486929e+62,62.0,8.000000e+00,0
1,1,0.693147,1.0,1,0.693147,1.000000,1,2.014102,1.103302,0.496499,...,0,False,0,2,-1,1,2.486929e+62,62.0,8.000000e+00,0
2,1,0.693147,1.0,2,1.098612,0.500000,4,3.016049,1.390299,0.331560,...,0,False,0,2,-1,1,3.885728e+08,8.0,7.019868e+00,0
3,1,0.693147,1.0,3,1.386294,0.333333,9,4.027806,1.614984,0.248274,...,0,False,0,2,-1,1,9.917391e-23,-23.0,9.917391e-17,0
4,1,0.693147,1.0,4,1.609438,0.250000,16,5.035311,1.797627,0.198597,...,0,False,0,2,-1,1,8.003509e-23,-23.0,8.003509e-17,0


1. Independent Particle Model (IPM)
Features to Add: Interaction energies between nucleons, mean field potential.

The Independent Particle Model (IPM) assumes that each nucleon (proton or neutron) in the nucleus moves independently in a common potential well created by all other nucleons. In this model, the effects of the interactions between individual nucleons are averaged out. The focus is primarily on the properties of individual nucleons rather than the collective behavior of the nucleus.

Features: Single Particle Energies, Occupancy Numbers for Each Energy Level, Qunatum Numbers

In [6]:
import numpy as np

# Constants
V0 = 50  # MeV
hbar = 197.3  # MeV·fm/c
m = 938.3  # MeV/c^2

# Calculate k using the principal quantum number (n)
df['k Protons_IP_Model'] = df['Last Principal Quantum Number'] * np.pi
df['k Neutrons_IP_Model'] = df['Last Principal Quantum Number'] * np.pi

# Calculate single particle energies for protons and neutrons
df['Single Particle Energy Protons_IP_Model'] = V0 + ((hbar ** 2) * (df['k Protons_IP_Model'] ** 2)) / (2 * m)
df['Single Particle Energy Neutrons_IP_Model'] = V0 + ((hbar ** 2) * (df['k Neutrons_IP_Model'] ** 2)) / (2 * m)

df['Occupancy Number Protons_IP_Model'] = 2 * (df['Last Principal Quantum Number'] ** 2)
df['Occupancy Number Neutrons_IP_Model'] = 2 * (df['Last Principal Quantum Number'] ** 2)


2. Liquid Drop Model - LDM

The Liquid Drop Model treats the nucleus as a drop of incompressible nuclear fluid. This model accounts for the gross properties of the nucleus, such as its energy, in terms of volume, surface, Coulomb, asymmetry, and pairing terms. It is often used to approximate binding energies and other macroscopic properties.

Features: Volume Temr, Surface Term, Coulomb Term, Asymmetry Term, Pairing Term

In [7]:


# Calculate Volume Term
df['Volume Term_LD_Model'] = df['Protons'] + df['Neutrons']

# Calculate Surface Term
df['Surface Term_LD_Model'] = df['Volume Term_LD_Model'] ** (2/3)

# Calculate Coulomb Term
df['Coulomb Term_LD_Model'] = (df['Protons'] * (df['Protons'] - 1)) / (df['Volume Term_LD_Model'] ** (1/3))

# Calculate Asymmetry Term
df['Asymmetry Term_LD_Model'] = ((df['Neutrons'] - df['Protons']) ** 2) / df['Volume Term_LD_Model']

# Calculate Pairing Term
df['Pairing Term_LD_Model'] = ((df['Protons'] % 2 == 0) & (df['Neutrons'] % 2 == 0)) | ((df['Protons'] % 2 != 0) & (df['Neutrons'] % 2 != 0))
df['Pairing Term_LD_Model'] = df['Pairing Term_LD_Model'].astype(int)


3. Fermi Gas Model - FGM

The Fermi Gas Model treats nucleons as non-interacting particles moving in a nuclear potential well, similar to free particles in a gas. The model is particularly useful for understanding the behavior of nucleons at high energies and temperatures. It relies heavily on Fermi energy and Fermi momentum to describe the state of the nucleus.

Features: Fermi Energy, Fermi Momentum, Nucleon Density

V is approximated with 1.33piR^3, where R=1.2A^.33

In [8]:
# Constants
pi = np.pi

# Calculate the radius of the nucleus (in femtometers)
df['Nucleus Radius_FG_Model'] = 1.2 * (df['Volume Term_LD_Model'] ** (1/3))

# Calculate the volume of the nucleus (in cubic femtometers)
df['Nucleus Volume_FG_Model'] = (4/3) * pi * (df['Nucleus Radius_FG_Model'] ** 3)

# Calculate nucleon density (in femtometers^-3)
df['Nucleon Density_FG_Model'] = df['Volume Term_LD_Model'] / df['Nucleus Volume_FG_Model']

# Calculate Fermi Energy (in MeV)
df['Fermi Energy_FG_Model'] = (hbar ** 2) / (2 * m) * (3 * pi ** 2 * df['Nucleon Density_FG_Model']) ** (2/3)

# Calculate Fermi Momentum (in MeV/c)
df['Fermi Momentum_FG_Model'] = hbar * ((3 * pi ** 2 * df['Nucleon Density_FG_Model']) ** (1/3))



4. Collective Model

The Collective Model is a hybrid approach that combines elements of the Independent Particle Model and the Liquid Drop Model to describe collective excitations in the nucleus, such as vibrations and rotations. This model is especially useful for understanding deformed nuclei and their collective behaviors.

Features: Deformation Parameters, Vibrational Quantum Numbers, Rotational Quantum Numbers, Collective Energy

In [9]:
# Step 1: Calculate Deformation Parameters
df['Beta Deformation_C_Model'] = df['Neutrons'] / df['Protons']
df['Gamma Deformation_C_Model'] = (df['Neutrons'] - df['Protons']) / df['Volume Term_LD_Model']

# Step 2: Derived Vibrational Quantum Numbers
df['Derived Vibrational Quantum Number_C_Model'] = 2 * df['Last Principal Quantum Number']
#df['Approximated Vibrational Quantum Number_C_Model'] = Number of Electrons in Outermost Shell -- ?????

# Step 3: Rotational Quantum Numbers (using the Last Azimuthal Quantum Number)
df['Rotational Quantum Number_C_Model'] = df['Last Azimuthal Quantum Number Encoded']

# Step 4: Calculate Collective Energy (as a weighted sum of vibrational and rotational quantum numbers)
w1, w2 = 1, 1  # Weights for simplicity
df['Collective Energy_C_Model'] = w1 * df['Derived Vibrational Quantum Number_C_Model'] + w2 * df['Rotational Quantum Number_C_Model']

6. Two-Center Shell Model - TCSM

The Two-Center Shell Model is used to describe nuclear systems where two separate potential wells are present, such as in nuclear reactions involving heavy ions or fission processes. In this model, each nucleus has its own set of energy levels, and the nucleons can tunnel between the two centers.

Features: Potential Energy for Each Center, Tunneling Probability, Combined Energy Levels, Relative Distance Between Centers, Interaction Energy, Angular Momentum Between Centers, Overlap Integral, Asymmetry Parameter

In [10]:
# Step 1: Center Separation
df['Center Separation_TCS_Model'] = np.cbrt(df['Atomic Weight'])

# Step 2: Calculate Shared Nucleons
# Approximating shared nucleons as a function of center separation and total nucleons
# For simplicity, we'll take a fraction (e.g., 10%) of the total nucleons as shared nucleons
shared_fraction = 0.1  # This is an arbitrary choice and can be adjusted
df['Shared Nucleons_TCS_Model'] = ((df['Protons'] + df['Neutrons']) * shared_fraction).astype(int)

df['Protons Center1_TCS_Model'] = df['Protons'] // 2  # Integer division to ensure whole numbers
df['Protons Center2_TCS_Model'] = df['Protons'] - df['Protons Center1_TCS_Model']  # The remaining protons go to the second center
df['Neutrons Center1_TCS_Model'] = df['Neutrons'] // 2
df['Neutrons Center2_TCS_Model'] = df['Neutrons'] - df['Neutrons Center1_TCS_Model']

# Step 3: Effective Individual Center Metrics
# Adjust the metrics for each center to account for the shared nucleons
df['Effective Protons Center1_TCS_Model'] = df['Protons Center1_TCS_Model'] - (df['Shared Nucleons_TCS_Model'] // 2)
df['Effective Protons Center2_TCS_Model'] = df['Protons Center2_TCS_Model'] - (df['Shared Nucleons_TCS_Model'] // 2)
df['Effective Neutrons Center1_TCS_Model'] = df['Neutrons Center1_TCS_Model'] - (df['Shared Nucleons_TCS_Model'] // 2)
df['Effective Neutrons Center2_TCS_Model'] = df['Neutrons Center2_TCS_Model'] - (df['Shared Nucleons_TCS_Model'] // 2)

# Step 4: Modified Interaction Terms
# Update the interaction terms to include the effects of shared nucleons
df['Modified Interaction Protons_TCS_Model'] = df['Effective Protons Center1_TCS_Model'] * df['Effective Protons Center2_TCS_Model']
df['Modified Interaction Neutrons_TCS_Model'] = df['Effective Neutrons Center1_TCS_Model'] * df['Effective Neutrons Center2_TCS_Model']
df['Modified Interaction Protons Neutrons_TCS_Model'] = df['Effective Protons Center1_TCS_Model'] * df['Effective Neutrons Center2_TCS_Model'] + df['Effective Protons Center2_TCS_Model'] * df['Effective Neutrons Center1_TCS_Model']

# Step 4: Deformation Parameters (using simplified approximations)
# Introducing Deformation Length and Deformation Width based on center separation and shared nucleons
# For simplicity, we'll scale these with center separation
df['Deformation Length_TCS_Model'] = df['Center Separation_TCS_Model'] * 1.1  # Scaled by a factor of 1.1 - Arbitrary
df['Deformation Width_TCS_Model'] = df['Center Separation_TCS_Model'] * 0.9  # Scaled by a factor of 0.9 - Arbitrary

# Step 4: Asymmetry Metrics
# Generating features for asymmetry between the two centers
df['Asymmetry Protons_TCS_Model'] = abs(df['Protons Center1_TCS_Model'] - df['Protons Center2_TCS_Model'])
df['Asymmetry Neutrons_TCS_Model'] = abs(df['Neutrons Center1_TCS_Model'] - df['Neutrons Center2_TCS_Model'])



7. No-Core Shell Model (NCSM)

The No-Core Shell Model (NCSM) is a modern ab initio approach to nuclear structure, starting from realistic nucleon-nucleon interactions without any phenomenological inputs. It does not truncate the Hilbert space to a model space and includes all possible configurations.

In [11]:
# Step 1: Interaction Terms for NCSM
# Calculating interaction terms between all active protons and neutrons
df['Interaction Protons_NCS_Model'] = df['Protons'] * (df['Protons'] - 1)  # Interaction among all protons
df['Interaction Neutrons_NCS_Model'] = df['Neutrons'] * (df['Neutrons'] - 1)  # Interaction among all neutrons
df['Interaction Protons Neutrons_NCS_Model'] = df['Protons'] * df['Neutrons']  # Interaction between protons and neutrons

# Step 2: Excitation Energy for NCSM
# Approximating excitation energy as a function of the total number of nucleons
# For simplicity, we'll take it to be proportional to the square of the total number of nucleons
df['Excitation Energy_NCS_Model'] = (df['Protons'] + df['Neutrons']) ** 2

# Step 3: Higher-Order Interactions for NCSM (Optional)
# Features capturing the interactions between three or more nucleons
# For simplicity, we'll take it to be proportional to the cube of the total number of nucleons
df['Higher Order Interactions_NCS_Model'] = (df['Protons'] + df['Neutrons']) ** 3


In [12]:
df.to_csv('nuclear_physics_final_df.csv')

8. Ab Initio Methods

Ab initio methods in nuclear physics aim to solve the many-body problem starting from first principles. These methods often use realistic interactions and do not rely on phenomenological models. Techniques like Quantum Monte Carlo, Coupled Cluster, and Density Functional Theory are examples of ab initio methods.